In [3]:
import mindspore as ms
import numpy as np

In [4]:
points = ms.Tensor(np.random.rand(9,4),ms.float64)

In [12]:
print(points[...,0])

[0.5920889  0.44426756 0.33231965 0.45367665 0.13016786 0.27614495
 0.94041053 0.36154926 0.81101449]


In [14]:
idx = ms.Tensor([0,3],ms.int32)
points[idx]

Tensor(shape=[2, 4], dtype=Float64, value=
[[5.92088902e-001, 7.98575706e-001, 7.92801691e-001, 4.01275562e-001],
 [4.53676647e-001, 5.37535793e-002, 8.83032206e-001, 4.71393508e-001]])

In [16]:
ss = ()

In [2]:
import mindspore.nn as nn
import mindspore.ops.operations as P
class ChannelShuffle(nn.Cell):
    def __init__(self):
        super(ChannelShuffle, self).__init__()
        self.reshape = P.Reshape()
        self.transpose = P.Transpose()
        self.shape = P.Shape()
    def construct(self, x):
        batchsize, num_channels, height, width = self.shape(x)
        x = self.reshape(x, (batchsize * num_channels // 2, 2, height * width,))
        x = self.transpose(x, (1, 0, 2,))
        x = self.reshape(x, (2, -1, num_channels // 2, height, width,))
        return x[0:1, :, :, :, :], x[-1:, :, :, :, :]

In [34]:
x = ms.Tensor(np.random.rand(10,24,320,320),ms.float32)

In [35]:
shuffle = ChannelShuffle()

In [36]:
x1, x2 = shuffle(x)

In [37]:
x1.shape

(1, 10, 12, 320, 320)

In [1]:
import math
import itertools as it
import numpy as np

In [2]:
class GeneratDefaultGridCellsII:
    def __init__(self):
        img_shape = [320, 320]
        strides = [8, 16, 32]
        feature_size = [40, 20, 10]
        anchor_size = np.array([8, 16, 32])
        aspect_ratio = 1.0

        fk = img_shape[0] / np.array(strides)
        scales = np.array([1.0])
        strides = np.array(strides)
        anchor_size = np.array(anchor_size)
        self.default_multi_level_grid_cells = []
        # config.feature_size = [40, 20, 10]
        for idex, feature_size in enumerate(feature_size):
            base_size = anchor_size[idex] / img_shape[0]
            size = base_size * scales[0]
            all_sizes = []
            w, h = size * math.sqrt(aspect_ratio), size / math.sqrt(aspect_ratio)
            all_sizes.append((h, w))
            for i, j in it.product(range(feature_size), repeat=2):
                for h, w in all_sizes:
                    cx, cy = (j + 0.5) / fk[idex], (i + 0.5) / fk[idex]
                    self.default_multi_level_grid_cells.append([cx,cy,h,w])
        def to_ltrb(cy, cx, h, w):
            return cy - h / 2, cx - w / 2, cy + h / 2, cx + w / 2

        self.default_multi_level_grid_cells_ltrb = np.array(tuple(to_ltrb(*i) for i in self.default_multi_level_grid_cells), dtype='float32')
        self.default_multi_level_grid_cells = np.array(self.default_multi_level_grid_cells, dtype='float32')


In [73]:
GeneratDefaultGridCellsII().default_multi_level_grid_cells

array([[0.0125  , 0.0125  , 0.003125, 0.003125],
       [0.0375  , 0.0125  , 0.003125, 0.003125],
       [0.0625  , 0.0125  , 0.003125, 0.003125],
       ...,
       [0.75    , 0.95    , 0.003125, 0.003125],
       [0.85    , 0.95    , 0.003125, 0.003125],
       [0.95    , 0.95    , 0.003125, 0.003125]], dtype=float32)

In [75]:
GeneratDefaultGridCellsII().default_multi_level_grid_cells_ltrb

array([[0.0109375, 0.0109375, 0.0140625, 0.0140625],
       [0.0359375, 0.0109375, 0.0390625, 0.0140625],
       [0.0609375, 0.0109375, 0.0640625, 0.0140625],
       ...,
       [0.7484375, 0.9484375, 0.7515625, 0.9515625],
       [0.8484375, 0.9484375, 0.8515625, 0.9515625],
       [0.9484375, 0.9484375, 0.9515625, 0.9515625]], dtype=float32)

In [27]:
y1, x1, y2, x2 = np.split(default_boxes_ltrb[:, :4], 4, axis=-1)

ValueError: array split does not result in an equal division

In [46]:
class GeneratDefaultBoxes():
    """
    Generate Default boxes for retinanet, follows the order of (W, H, archor_sizes).
    `self.default_boxes` has a shape of [archor_sizes, H, W, 4], the last dimension is [y, x, h, w].
    `self.default_boxes_ltrb` has a shape as `self.default_boxes`, the last dimension is [y1, x1, y2, x2].
    """
    def __init__(self):
        img_shape = [600, 600]
        steps = [8, 16, 32, 64, 128]
        anchor_size = [32, 64, 128, 256, 512]
        feature_size = [75, 38, 19, 10, 5]
        aspect_ratios =  [[0.5, 1.0, 2.0], [0.5, 1.0, 2.0], [0.5, 1.0, 2.0], [0.5, 1.0, 2.0], [0.5, 1.0, 2.0]]

        fk = img_shape[0] / np.array(steps)
        scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])
        anchor_size = np.array(anchor_size)
        self.default_boxes = []
        for idex, feature_size in enumerate(feature_size):
            base_size = anchor_size[idex] / img_shape[0]
            size1 = base_size*scales[0]
            size2 = base_size*scales[1]
            size3 = base_size*scales[2]
            all_sizes = []
            for aspect_ratio in aspect_ratios[idex]:
                w1, h1 = size1 * math.sqrt(aspect_ratio), size1 / math.sqrt(aspect_ratio)
                all_sizes.append((h1, w1))
                w2, h2 = size2 * math.sqrt(aspect_ratio), size2 / math.sqrt(aspect_ratio)
                all_sizes.append((h2, w2))
                w3, h3 = size3 * math.sqrt(aspect_ratio), size3 / math.sqrt(aspect_ratio)
                all_sizes.append((h3, w3))

            # assert len(all_sizes) == config.num_default[idex]

            # 第一个归一化操作, 按照feature_size 除以
            # 为5个尺度的特征图，每个点上，铺上9个Anchor，铺满全图
            for i, j in it.product(range(feature_size), repeat=2):
                for h, w in all_sizes:
                    cx, cy = (j + 0.5) / fk[idex], (i + 0.5) / fk[idex]
                    self.default_boxes.append([cy, cx, h, w])

        # 偏移变换到左上角和右下角
        def to_ltrb(cy, cx, h, w):
            return cy - h / 2, cx - w / 2, cy + h / 2, cx + w / 2

        # For IoU calculation
        self.default_boxes_ltrb = np.array(tuple(to_ltrb(*i) for i in self.default_boxes), dtype='float32')
        self.default_boxes = np.array(self.default_boxes, dtype='float32')


In [48]:
GeneratDefaultBoxes().default_boxes.shape

(67995, 4)

In [50]:
GeneratDefaultBoxes().default_boxes_ltrb.shape

(67995, 4)

In [6]:
import mindspore as ms
idx = ms.Tensor([[1,2,7,4],[1,2,3,4]])

In [7]:
idx.shape

(2, 4)

In [82]:
idx.view(-1)

Tensor(shape=[8], dtype=Int32, value= [1, 2, 7, 4, 1, 2, 3, 4])

In [14]:
idx>1

Tensor(shape=[2, 4], dtype=Bool, value=
[[False,  True,  True,  True],
 [False,  True,  True,  True]])

In [15]:
import mindspore.ops.operations as P

In [16]:
P.ExpandDims()(idx, 0)

Tensor(shape=[1, 2, 4], dtype=Int32, value=
[[[1, 2, 7, 4],
  [1, 2, 3, 4]]])